# Websockets

- WebSocket es un protocolo de comunicación que proporciona un canal full-duplex (bidireccional simultáneo) persistente sobre una única conexión TCP.   
- Permite la comunicación en tiempo real entre cliente (por ejemplo, un navegador web) y servidor con baja sobrecarga de la red, evitando la necesidad de realizar múltiples solicitudes HTTP o HTTPS.
- HTTP tradicional es un modelo de request-response → el cliente siempre inicia la comunicación.
   - Para aplicaciones en tiempo real (chats, juegos online, trading), puede usar técnicas como:
     - Polling: puede enviar peticiones periódicas, pero es ineficiente.
     - Long Polling: puede mantener la conexión abierta hasta que el servidor responda, costoso en recursos.
- WebSocket (RFC 6455 - 2011) soluciona estos problemas proporcionando una conexión persistente eficiente.

### Características de Websockets
Nota: donde dice HTTP puede ser también HTTPS.

- Permite una comunicación full-duplex
   - Cliente y servidor pueden enviar mensajes en cualquier momento sin necesidad de iniciar una nueva conexión.
- Tiene un bajo consumo de recursos
   - Realiza un handshake inicial usando HTTP.
     - Reduce el overhead de cabeceras.
- Latencia baja
   - Permite aplicaciones en tiempo real.
- Compatibilidad
   - Puede consideranrse una limitación leve
     - Requiere soporte tanto del navegador como del servidor.

### Cómo funciona

- Handshake
   - Cliente abre conexión HTTP tradicional al servidor (puerto 80 o 443).
   - Cliente envía una cabecera especial:

- Si el servidor acepta, responde que se cambia del protocolo HTTP al WebSocket sobre la misma conexión TCP.

- Comunicación
  - WebSocket establecido.
  - Intercambio de mensajes binario o texto,
  - Usa tramas compactas.
  - No se requier handshake hasta que la conexión se cierre.

### Arquitectura

- Cliente
   - Navegador web
   - Usa JavaScript para crear y gestionar la conexión.

- Servidor
   - Múltiples opciones
     - Node.js
     - Python (con websockets, Flask-SocketIO), 
     - Java, etc.
   - Interacción
     - Puede enviar datos al cliente sin que éste lo solicite explícitamente (*push*).

### Seguridad

- WSS (WebSocket Secure)
  - Equivalente a HTTPS.
  - Encripta la conexión usando TLS.
- Problemas usuales
  - Cross-Site WebSocket Hijacking (CSWSH).
  - Falta de autenticación en el canal WebSocket.
- Reconendaciones
  - Utilizar WSS siempre que sea posible.
  - Implementar mecanismos de autenticación adicionales.
  - Controlar el origen de las conexiones (Origin header).

### Casos de Uso de WebSockets

- Chats en tiempo real (WhatsApp Web, Slack)
- Juegos online multijugador
- Trading de alta frecuencia (actualización de precios)
- Control remoto de dispositivos
- Colaboración en tiempo real (editores de texto, pizarras colaborativas)

### Comparación con otros protocolos

| Protocolo| Tipo de Comunicación | Overhead | Persistencia | Latencia |
|:-------------|:---------------:|:-------------:|:-------------:|:-------------:|
HTTP | Unidireccional | Alto | No | Alta
HTTP Long Polling | Semidúplex | Medio | Parcial | Media
WebSocket | Full-duplex | Bajo | Sí | Baja
Server-Sent Events (SSE) | Solo servidor → cliente | Bajo | Sí | Baja

### Limitaciones 

- No todos los proxies/firewalls manejan bien conexiones WebSocket.
- Mantenimiento de conexiones persistentes es costoso en servidores de gran escala.
  - Clusters o balanceo de carga WebSocket-aware puede reducir costos.
- Clientes móviles las conexiones persistentes pueden impactar en el uso de batería.

## Como interactua websockets a bajo nivel en su relacion son socket(), y el stack de TCP/IP

- Recordemos que socket() es una llamada al sistema (system call) que permite crear un descriptor de comunicación para transmitir datos entre procesos (locales o remotos).
- socket() NO es un "WebSocket".
- socket() es una abstracción de nivel de transporte o inferior.

### ¿Qué ocurre a bajo nivel cuando usamos WebSocket?

- Conexión inicial
  - Cliente (navegador) usa socket() internamente para abrir una conexión TCP al servidor.
  - Por tuplas de IP y puerto 80 ó 443.
  - Hace un handshake HTTP solicitando un "cambio de protocolo" a websocket:
  - **Esto todavía es HTTP normal sobre TCP/IP**
  - El Servidor (también usando un socket()) recibe esta solicitud y responde con 101 **"Switching Protocols"**.
  - Ahora ambos extremos "acuerdan" que, sobre esa misma conexión TCP, ya no usarán HTTP, sino el protocolo WebSocket.   
- Comunicación de datos
  - WebSocket define su propio encapsulado.
  - Cada mensaje WebSocket se encapsula en tramos (estructuras binarias) que se transmiten como payloads TCP.
  - TCP se encarga de que estos datos lleguen ordenados y sin pérdidas.
  - El socket TCP que fue abierto sigue activo hasta que se cierra explícitamente.
  - WebSocket NO reemplaza a TCP, sino que se monta encima de TCP usando un framing propio.
  - Desde el punto de vista del sistema operativo, sigue siendo una conexión TCP.
  - Los paquetes TCP que se envían llevan en su contenido las tramas WebSocket.

Elemento | Función |
|:-------------|:---------------:|
socket() | Crea descriptor de comunicación TCP.
TCP/IP stack | Garantiza entrega, orden y conexión.
WebSocket API | Encapsula datos, maneja protocolo a nivel aplicación.

- Cuando se envía un mensaje
  - Se empaqueta según la especificación WebSocket
  - 2 bytes de encabezado mínimo.
  - Máscara de 4 bytes (si es cliente → servidor).
  - Payload de datos

| Campo | Contenido |
|:-------------|:---------------:|
FIN (1 bit) | 1 (última trama)
Opcode (4 bits) | 0x1 (texto)
Máscara (1 bit) | 1 (cliente → servidor)
Longitud Payload | 4 (por ejemplo, "Hola")
Máscara (4 bytes) | e.g., 0x37fa213d
Payload | Datos enmascarados ("Hola" encriptado con la máscara)

**Esta trama se escribe directamente en el socket TCP**.

- WebSocket requiere TCP
  - Necesita garantía de entrega.
  - Necesita orden (los mensajes llegan como se enviaron).
  - Necesita conexión persistente (estado entre cliente y servidor).
  - Con UDP implicaría pérdida y reordenación, lo cual rompería las garantías que WebSocket necesita.

### ¿ Cómo sabe tcp que se trata de un paquete websocket ?

In [ ]:
- TCP no lo sabe.
- TCP es agnóstico (no entiende) del contenido que transporta.
- TCP no sabe si transporta: HTTP, WebSocket, FTP, SSH, TLS.

### ¿Quién interpreta que es WebSocket?
- La aplicación que recibe los datos (por ejemplo, el servidor WebSocket).
- Sabe que en esa conexión se hizo un handshake de upgrade HTTP → WebSocket.
- Sabe que, desde ese momento, los datos que llegan están en formato de tramas WebSocket.
- Decodifica esas tramas siguiendo la RFC 6455 (norma de WebSocket).
- El acuerdo que vimos antes lo hacen las aplicaciones, no TCP.
- TCP sigue ciego, simplemente transporta los bytes.
- La capa de aplicación (navegador y servidor) saben interpretar esos bytes como tramas WebSocket.
- TCP no agrega ningún identificador que indique que el contenido es WebSocket.

### Entonces, ¿cómo se sabe que es WebSocket?
- Por convención de puertos.
  - WebSocket comúnmente se inicia sobre:
  - Puerto 80 (sin TLS)
  - Puerto 443 (con TLS/WSS)
- Por el contenido de los datos en la conexión.
  - El handshake HTTP inicial que solicita cambio de protocolo
  - Websocket es lo que define que en adelante se hablará WebSocket sobre TCP.
- Luego de ese acuerdo a nivel de aplicación, los datos se interpretan como frames WebSocket.

### ¿Y si alguien se conecta al puerto 80 o 443 sin usar WebSocket?
- Se puede.
- El servidor debe interpretar el contenido.
- Si el handshake correcto no ocurre, no se cambia el protocolo a WebSocket y la conexión puede ser rechazada o seguir como HTTP.
- El puerto es solo una convención, pero el contenido define la comunicación.
- Ejemplo: Puerto 80 (HTTP)
  - Puerto 80 es convencionalmente usado para HTTP.
  - Cuando se inicia una conexión a puerto 80, normalmente se espera hablar HTTP.
  - WebSocket (ws://) comienza como HTTP → envía un GET con Upgrade: websocket.
  - Si el servidor acepta el cambio, a partir de ahí hablan WebSocket en la misma conexión TCP.
- Entonces: **Puerto 80 = HTTP o WebSocket, según el contenido.**
- Ejemplo: Puerto 443 (HTTPS)
  - Puerto 443 es convencionalmente usado para HTTPS.
  - HTTPS significa HTTP sobre TLS/SSL (encriptado).
  - Cuando se inicia conexión al puerto 443, primero hay un handshake TLS (negociación de encriptado).
  - Después, dentro del túnel TLS, se usa HTTP normal o WebSocket seguro (wss://).
- Entonces: **Puerto 443 = HTTPS o WSS, otra vez depende del contenido después del TLS.**

## Servidores que soportan websockets

| Servidor Web | ¿Soporta WebSocket? | Notas | 
|:-------------|:---------------:|:---------------:|
Nginx | Sí (como proxy) | Desde la versión 1.3.x permite proxy WebSocket. Nginx no termina WebSockets, solo los pasa a la app backend.
Apache HTTP Server | Sí (con módulo) | Requiere módulos como mod_proxy_wstunnel para hacer de proxy WebSocket.
Caddy | Sí (nativo) | Soporte de WebSocket directo, pensado para HTTPS automático y WebSocket moderno.
Node.js (con Express, Koa, etc.) | Sí (nativo) | Node.js + librerías como ws, socket.io manejan WebSocket directo, sin necesidad de proxy.
HAProxy | Sí (como proxy) | Desde HAProxy 1.5, soporte explícito para WebSockets pasando correctamente Upgrade y Connection.
Tomcat (Java) | Sí (nativo) | Desde Tomcat 7, soporte para WebSocket según la especificación JSR 356.
Jetty (Java) | Sí (nativo) | Soporte completo de WebSockets. Usado mucho en aplicaciones Java.
Microsoft IIS | Sí (con WebSocket Protocol Module) | Desde IIS 8.0 en Windows Server 2012, WebSocket está disponible, pero requiere habilitarlo manualmente.
Traefik | Sí (nativo) | Balanceador moderno que soporta WebSocket automáticamente. Ideal para microservicios y Docker.

- Nginx
  - Funciona como proxy que reenvía WebSocket a un backend (como Node.js, Flask, etc.).
  - Asegurar de incluir los encabezados Upgrade y Connection.
- Apache HTTPD:
  - Usar mod_proxy + mod_proxy_wstunnel.
  - Similar a Nginx, solo pasa WebSockets al backend.
- Node.js:
  - Maneja WebSockets directamente en la aplicación.
  - Librerías más usuales:
    - ws (bajo nivel)
    - socket.io (nivel alto, eventos)


### ¿Qué pasa si el servidor no soporta WebSocket?
- No puede manejar Upgrade -> conexión WebSocket fallará.
- El cliente recibe un error de conexión.
- Se puede solucionar con un proxy (como Nginx o HAProxy) que pase la conexión a un backend que sí lo entienda.

## Ejemplo de cliente servidor usando websockets (un chat) usando node.js en servidor y flask en cliente

### Instalar node.js

### Instalar la librería ws en Node.js:

Servidor WebSocket en Node.js (server.js):

In [1]:
%%writefile server.js
const WebSocket = require('ws');

// Creamos un servidor WebSocket en el puerto 8080
const wss = new WebSocket.Server({ port: 8080 });

wss.on('connection', function connection(ws) {
  console.log('Cliente conectado.');

  // Cuando recibe un mensaje de un cliente
  ws.on('message', function incoming(message) {
    console.log('Recibido: %s', message);

    // Responder al cliente
    ws.send(`Servidor recibió: ${message}`);
  });

  // Enviar un mensaje al cliente apenas conecta
  ws.send('Bienvenido al chat WebSocket!');
});


Writing server.js


### Ejecutar el servidor desde terminal

### Para probar conexión ws

### Cliente Flask (Cliente WebSocket)

Instalar la librería de WebSocket para Python:

pip install websocket-client flask

Creamos el cliente Flask (client.py):

In [3]:
%%writefile client.py
from flask import Flask, render_template, request
from websocket import WebSocketApp
import threading
import time

# Crear aplicación Flask
app = Flask(__name__)

# WebSocket global
ws = None
ws_url = "ws://localhost:8080"  # URL del servidor WebSocket

# -------- FUNCIONES PARA MANEJAR EVENTOS WEBSOCKET -------- #

def on_message(ws, message):
    print(f"[Mensaje del servidor] {message}")

def on_error(ws, error):
    print(f"[Error de WebSocket] {error}")

def on_close(ws, close_status_code, close_msg):
    print("[WebSocket cerrado]")

def on_open(ws):
    print("[WebSocket abierto]")
    ws.send("Hola desde Flask WebSocket Client")

# -------- FUNCIONES PARA INICIAR WEBSOCKET -------- #

def start_websocket():
    global ws
    ws = WebSocketApp(
        ws_url,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close,
        on_open=on_open
    )
    # Este método bloquea, por eso lo ponemos en un hilo
    ws.run_forever(ping_interval=20, ping_timeout=10)

# -------- RUTAS FLASK -------- #

@app.route('/')
def index():
    return '''
        <h1>Enviar mensaje al servidor WebSocket</h1>
        <form action="/send" method="post">
            <input type="text" name="message" placeholder="Escribe tu mensaje">
            <input type="submit" value="Enviar">
        </form>
    '''

@app.route('/send', methods=['POST'])
def send():
    message = request.form['message']
    if ws and ws.sock and ws.sock.connected:
        ws.send(message)
        return f"Mensaje enviado: {message}<br><a href='/'>Enviar otro</a>"
    else:
        return "Error: WebSocket no está conectado.<br><a href='/'>Reintentar</a>"

# -------- MAIN -------- #

if __name__ == '__main__':
    # Lanzar WebSocket en hilo aparte
    websocket_thread = threading.Thread(target=start_websocket)
    websocket_thread.daemon = True
    websocket_thread.start()

    # Lanzar Flask
    app.run(port=5000)


Overwriting client.py


Ejecutar desde terminal 

python client.py

- El servidor Node.js escucha conexiones WebSocket.
- El cliente Flask se conecta como cliente WebSocket, usando websocket-client.
  - Escribir mensajes desde la web de Flask (http://localhost:5000).
  - Los mensajes se envían al servidor Node.js.
  - El servidor responde y el cliente Flask imprime la respuesta en la terminal.